---
title: "Word Embeddings"
date: "2024-02-12"
categories: [Colab]
image: https://colab.research.google.com/img/colab_favicon_256px.png
---

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Colab

<a href="https://colab.research.google.com/github/aotal/ImagenMedica/blob/master/posts/code/word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Word Embeddings*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aotal/ImagenMedica/blob/master/posts/code/word_embeddings.ipynb)

Este tutorial es una introducción a los word embeddings. Entrenarás tus propios word embeddings utilizando un modelo simple de Keras para una tarea de clasificación de sentimientos y luego los visualizarás en el [Embedding Projector](http://projector.tensorflow.org) (como se muestra en la imagen a continuación).

<img src="https://github.com/tensorflow/text/blob/master/docs/tutorials/images/embedding.jpg?raw=1" alt="Screenshot of the embedding projector" width="400"/>

## Representación de texto como números

Los modelos de aprendizaje automático toman vectores (matrices de números) como entrada. Cuando se trabaja con texto, lo primero que debes hacer es idear una estrategia para convertir cadenas en números (o "vectorizar" el texto) antes de introducirlo en el modelo. En esta sección, veremos tres estrategias para hacerlo.

### Codificaciones One-hot

Como primera idea, podrías codificar cada palabra en tu vocabulario con "one-hot". Considera la frase "El gato se sentó en el tapete". El vocabulario (o palabras únicas) en esta frase es (gato, tapete, en, sentó, el). Para representar cada palabra, crearás un vector cero con una longitud igual al vocabulario y luego colocarás un uno en el índice que corresponde a la palabra. Este enfoque se muestra en el siguiente diagrama.

Para crear un vector que contenga la codificación de la frase, podrías concatenar los vectores one-hot para cada palabra.

Punto clave: Este enfoque es ineficiente. Un vector codificado con one-hot es disperso (es decir, la mayoría de los índices son cero). Imagina que tienes 10000 palabras en el vocabulario. Para codificar cada palabra con one-hot, crearías un vector donde el 99.99% de los elementos son cero.

<img src="https://www.tensorflow.org/static/text/guide/images/one-hot.png" alt="Diagrama de codificación one-hot" width="400" />

### Codifica cada palabra con un número único

Un segundo enfoque que podrías intentar es codificar cada palabra usando un número único. Continuando con el ejemplo anterior, podrías asignar 1 a "gato", 2 a "tapete", y así sucesivamente. Luego, podrías codificar la frase "El gato se sentó en el tapete" como un vector denso como [5, 1, 4, 3, 5, 2]. Este enfoque es eficiente. En lugar de un vector disperso, ahora tienes uno denso (donde todos los elementos están llenos).

Sin embargo, hay dos desventajas en este enfoque:

- La codificación de enteros es arbitraria (no captura ninguna relación entre palabras).

- Una codificación de enteros puede ser difícil de interpretar para un modelo. Un clasificador lineal, por ejemplo, aprende un solo peso para cada característica. Debido a que no existe una relación entre la similitud de dos palabras y la similitud de sus codificaciones, esta combinación de peso de característica no tiene sentido.

### *Word embeddings*

Los word embeddings nos brindan una forma de usar una representación eficiente y densa en la que palabras similares tienen una codificación similar. Es importante destacar que no tienes que especificar esta codificación manualmente. Una incrustación (embedding) es un vector denso de valores de punto flotante (la longitud del vector es un parámetro que especificas). En lugar de especificar los valores para la incrustación manualmente, son parámetros entrenables (pesos aprendidos por el modelo durante el entrenamiento, de la misma manera que un modelo aprende pesos para una capa densa). Es común ver word embeddings que son de 8 dimensiones (para conjuntos de datos pequeños), hasta 1024 dimensiones cuando se trabaja con conjuntos de datos grandes. Una incrustación de mayor dimensión puede capturar relaciones detalladas entre palabras, pero se necesitan más datos para aprender.

<img src="https://www.tensorflow.org/text/guide/images/embedding2.png?hl=es-419" alt="Diagram of an *embedding*" width="400"/>

Arriba hay un diagrama para una incrustación de palabras (word embedding). Cada palabra se representa como un vector de 4 dimensiones de valores de punto flotante. Otra forma de pensar en una incrustación (embedding) es como una "tabla de búsqueda". Después de que se han aprendido estos pesos, puedes codificar cada palabra buscando el vector denso al que corresponde en la tabla.

## Preparación

In [ ]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

### Descargar el conjunto de datos IMDb
Utilizaremos el [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) a lo largo del tutorial. Entrenarás un modelo clasificador de sentimientos en este conjunto de datos y en el proceso aprenderás word embeddings desde cero. Para leer más sobre cómo cargar un conjunto de datos desde cero, consulta el [Loading text tutorial](https://www.tensorflow.org/tutorials/load_data/text).  

Descarga el conjunto de datos usando la utilidad de archivos de Keras y echa un vistazo a los directorios.

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

Echa un vistazo al directorio train/. Tiene carpetas pos y neg con reseñas de películas etiquetadas como positivas y negativas, respectivamente. Utilizarás las reseñas de las carpetas pos y neg para entrenar un modelo de clasificación binaria.

In [ ]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

The `train` directory also has additional folders which should be removed before creating training dataset.

In [ ]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

A continuación, crea un tf.data.Dataset usando tf.keras.utils.text_dataset_from_directory. Puedes leer más sobre el uso de esta utilidad en este [text classification tutorial](https://www.tensorflow.org/tutorials/keras/text_classification).

Utiliza el directorio train para crear conjuntos de datos de entrenamiento y validación con una división del 20% para la validación.

In [ ]:
batch_size = 1024
seed = 123
train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

Echa un vistazo a algunas reseñas de películas y sus etiquetas (1: positivo, 0: negativo) del conjunto de datos de entrenamiento.


In [ ]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(5):
    print(label_batch[i].numpy(), text_batch.numpy()[i])

### Configurar el conjunto de datos para el rendimiento

Estos son dos métodos importantes que debes usar al cargar datos para asegurarte de que la E/S no se bloquee.

`.cache()` mantiene los datos en la memoria después de que se cargan desde el disco. Esto asegurará que el conjunto de datos no se convierta en un cuello de botella mientras se entrena tu modelo. Si tu conjunto de datos es demasiado grande para caber en la memoria, también puedes usar este método para crear una caché en disco de alto rendimiento, que es más eficiente de leer que muchos archivos pequeños.

`.prefetch()` superpone el preprocesamiento de datos y la ejecución del modelo durante el entrenamiento.

Puedes obtener más información sobre ambos métodos, así como sobre cómo almacenar en caché los datos en el disco en la  [data performance guide](https://www.tensorflow.org/guide/data_performance).

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Usando la capa Embedding

Keras facilita el uso de word embeddings. Echa un vistazo a la capa [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding).

La capa Embedding se puede entender como una tabla de búsqueda que mapea desde índices enteros (que representan palabras específicas) a vectores densos (sus incrustaciones - embeddings). La dimensionalidad (o ancho) de la incrustación es un parámetro con el que puedes experimentar para ver qué funciona bien para tu problema, de la misma manera que experimentarías con la cantidad de neuronas en una capa densa.


In [ ]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(1000, 5)

Cuando creas una capa Embedding, los pesos para la incrustación se inicializan aleatoriamente (como cualquier otra capa). Durante el entrenamiento, se ajustan gradualmente mediante retropropagación. Una vez entrenados, los word embeddings aprendidos codificarán aproximadamente las similitudes entre las palabras (ya que se aprendieron para el problema específico en el que se entrena tu modelo).

Si pasas un entero a una capa Embedding, el resultado reemplaza cada entero con el vector de la tabla de incrustaciones:

In [ ]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

Para problemas de texto o secuencia, la capa Embedding toma un tensor 2D de enteros, de forma `(muestras, longitud_de_secuencia)`, donde cada entrada es una secuencia de enteros. Puede incrustar secuencias de longitudes variables. Podrías alimentar a la capa de incrustación anterior lotes con formas `(32, 10)` (lote de 32 secuencias de longitud 10) or `(64, 15)` (lote de 64 secuencias de longitud 15).

El tensor devuelto tiene un eje más que la entrada, los vectores de incrustación se alinean a lo largo del nuevo último eje. Pásale un lote de entrada `(2, 3)` y la salida es `(2, 3, N)`


In [ ]:
result = embedding_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))
result.shape

Cuando se le da un lote de secuencias como entrada, una capa Embedding devuelve un tensor de punto flotante 3D, de forma `(muestras, longitud_de_secuencia, dimensionalidad_de_la_incrustación)`. Para convertir de esta secuencia de longitud variable a una representación fija, hay una variedad de enfoques estándar. Podrías usar una RNN, Atención o una capa de agrupación antes de pasarla a una capa Densa. Este tutorial utiliza la agrupación porque es la más simple. El[Text Classification with an RNN](https://www.tensorflow.org/text/tutorials/text_classification_rnn) es un buen siguiente paso.

## Preprocesamiento de texto




A continuación, define los pasos de preprocesamiento del conjunto de datos necesarios para tu modelo de clasificación de sentimientos. Inicializa una capa TextVectorization con los parámetros deseados para vectorizar las reseñas de películas. Puedes obtener más información sobre el uso de esta capa en el tutorial [Text Classification](https://www.tensorflow.org/tutorials/keras/text_classification).

In [ ]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

## Crea un modelo de clasificación

Utiliza la [Keras Sequential API](https://www.tensorflow.org/guide/keras/sequential_model) para definir el modelo de clasificación de sentimientos. En este caso, es un modelo de estilo "bolsa de palabras continua".
* La capa [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) transforma cadenas en índices de vocabulario. Ya has inicializado `vectorize_layer` como una capa TextVectorization y has creado su vocabulario llamando a `adapt` en `text_ds`. Ahora `vectorize_layer` se puede utilizar como la primera capa de tu modelo de clasificación de extremo a extremo, alimentando cadenas transformadas en la capa.
* La capa [`Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) toma el vocabulario codificado en enteros y busca el vector de incrustación (embedding) para cada índice de palabra. Estos vectores se aprenden a medida que se entrena el modelo. Los vectores agregan una dimensión a la matriz de salida. Las dimensiones resultantes son: `(batch, sequence, embedding)`.

* La capa [`GlobalAveragePooling1D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling1D) devuelve un vector de salida de longitud fija para cada ejemplo promediando sobre la dimensión de la secuencia. Esto permite que el modelo maneje la entrada de longitud variable, de la manera más simple posible.

* El vector de salida de longitud fija se canaliza a través de una capa completamente conectada ([`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)) con 16 unidades ocultas.

* La última capa está densamente conectada con un solo nodo de salida.

Precaución: Este modelo no utiliza enmascaramiento, por lo que el relleno de ceros se utiliza como parte de la entrada y, por lo tanto, la longitud del relleno puede afectar la salida. Para solucionar esto, consulta [masking and padding guide](https://www.tensorflow.org/guide/keras/masking_and_padding).

In [ ]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

## Compila y entrena el modelo

Utilizarás [TensorBoard](https://www.tensorflow.org/tensorboard) ara visualizar métricas, incluidas la pérdida y la precisión. Crea un tf.keras.callbacks.TensorBoard.

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

Compila y entrena el modelo utilizando el optimizador Adam y la pérdida BinaryCrossentropy.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

Con este enfoque, el modelo alcanza una precisión de validación de alrededor del 78% (ten en cuenta que el modelo está sobreajustado ya que la precisión del entrenamiento es mayor).

Nota: Tus resultados pueden ser un poco diferentes, dependiendo de cómo se inicializaron aleatoriamente los pesos antes de entrenar la capa Embedding.

Puedes consultar el resumen del modelo para obtener más información sobre cada capa del modelo.

In [ ]:
model.summary()

Visualiza las métricas del modelo en TensorBoard.

In [ ]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir logs

![embeddings_classifier_accuracy.png](https://www.tensorflow.org/text/guide/images/embeddings_classifier_accuracy.png?hl=es-419)

## Recupere las incrustaciones de palabras entrenadas y guárdelas en el disco

A continuación, recupere las incrustaciones de palabras aprendidas durante el entrenamiento. Las incrustaciones son pesos de la capa de incrustación en el modelo. La matriz de ponderaciones tiene la forma (vocab_size, embedding_dimension) .



Obtenga los pesos del modelo usando get_layer() y get_weights() . La función get_vocabulary() proporciona el vocabulario para crear un archivo de metadatos con un token por línea.

In [ ]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

Escriba los pesos en el disco. Para utilizar el proyector de incrustaciones , deberá cargar dos archivos en formato separado por tabulaciones: un archivo de vectores (que contiene la incrustación) y un archivo de metadatos (que contiene las palabras).

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

Si está ejecutando este tutorial en Colaboratory , puede usar el siguiente fragmento para descargar estos archivos a su máquina local (o usar el explorador de archivos, Ver -> Tabla de contenido -> Explorador de archivos ).

In [ ]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

## Visualiza las incrustaciones

Para visualizar las incrustaciones, súbalas al proyector de incrustaciones.

Abra el [Embedding Projector](http://projector.tensorflow.org/) (esto también se puede ejecutar en una instancia local de TensorBoard)).

* Haga clic en "Cargar datos".

* Cargue los dos archivos que creó anteriormente: `vecs.tsv` and `meta.tsv`.

Ahora se mostrarán las incrustaciones que ha entrenado. Puede buscar palabras para encontrar a sus vecinos más cercanos. Por ejemplo, intente buscar "hermoso". Es posible que vea vecinos como "maravilloso".

Nota: Experimentalmente, es posible que pueda producir incrustaciones más interpretables mediante el uso de un modelo más simple. Intente eliminar la capa Dense(16) , vuelva a entrenar el modelo y visualice las incrustaciones nuevamente.

Nota: Por lo general, se necesita un conjunto de datos mucho más grande para entrenar incrustaciones de palabras más interpretables. Este tutorial utiliza un pequeño conjunto de datos de IMDb con fines de demostración.






## Próximos pasos

Este tutorial le ha mostrado cómo entrenar y visualizar incrustaciones de palabras desde cero en un pequeño conjunto de datos.

* Para entrenar incrustaciones de palabras usando el algoritmo de Word2Vec, pruebe el tutorial de[Word2Vec](https://www.tensorflow.org/tutorials/text/word2vec) .

* Para obtener más información sobre el procesamiento de texto avanzado, lea el [modelo de Transformer para la comprensión del lenguaje](https://www.tensorflow.org/text/tutorials/transformer).